In [3]:
import os
import fnmatch
import glob
import numpy
import pickle
import copy
from numpy import genfromtxt,min,mean,max,std,array,shape,nanmin
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [4]:
open_ml_ids= [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20,21,22,23,24,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,47,48,49,50,52,53,54,55,57,58,59,60,2065,2073,2074,2075,2078,2079,3022]
#Need to rerun for nvb
open_ml_ids.remove(7)
#Not enough arms trained on full train set for nvb_hyperband
open_ml_ids.remove(6)
open_ml_ids.remove(26)
open_ml_ids.remove(34)
open_ml_ids.remove(38)
open_ml_ids.remove(2078)
#Do not converge within 10 mins
open_ml_ids.remove(2065)
open_ml_ids.remove(2075)
#dataset_ids=[item for sublist in open_ml_ids for item in sublist]

datasets_keep = open_ml_ids
len(datasets_keep)
dataset_max_train_size=pickle.load(open('../dataset_max_train_size.pkl','rb'))

In [56]:
def calculate_avg(errors,test_errors,durations,intervals,dataset,normalization=None):
    min_errors=[]
    min_test_errors=[]
    sum_durations=[]
    interval_errors=[]
    interval_test_errors=[]
    for t in range(len(errors)):
        err=errors[t]
        test_err=test_errors[t]
        temp=numpy.ones(len(err))
        temp_test=numpy.ones(len(err))
        temp[0]=numpy.nanmin([1,err[0]])
        temp_test[0]=numpy.nanmin([1,test_err[0]])
        for iter in range(1,len(err)):
            if numpy.nanmin([err[iter],temp[iter-1]])<temp[iter-1]:
                temp[iter]=err[iter]
                temp_test[iter]=test_err[iter]
            else:
                temp[iter]=temp[iter-1]
                temp_test[iter]=temp_test[iter-1]                
        min_errors.append(temp)
        min_test_errors.append(temp_test)
    for dur in durations:
        temp = [sum(dur[0:(i+1)]) for i in range(len(dur))]
        if normalization is not None:
            temp=numpy.array(temp)/normalization[dataset]
        sum_durations.append(temp)
    for trial in range(len(sum_durations)):
        min_err=min_errors[trial]
        min_test_err=min_test_errors[trial]
        sum_dur=sum_durations[trial]
        temp=[]
        temp_test=[]
        for t in intervals:
            indices=[ n for n,i in enumerate(sum_dur) if i<=t ]
            if len(indices)>0:
                temp.append(min_err[indices[-1]])
                temp_test.append(min_test_err[indices[-1]])
            else:
                temp.append(1)
                temp_test.append(1)
        interval_errors.append(temp)
        interval_test_errors.append(temp_test)
    avg_interval_err = numpy.mean(interval_errors,axis=0)
    avg_interval_test_err = numpy.mean(interval_test_errors,axis=0)
    std_interval_err = numpy.std(interval_errors,axis=0)
    std_interval_test_err = numpy.std(interval_test_errors,axis=0)
    return len(errors), avg_interval_err, std_interval_err, avg_interval_test_err, std_interval_test_err

In [7]:
def calculate_avg_iters(errors,test_errors):
    min_errors=[]
    min_test_errors=[]
    sum_durations=[]
    interval_errors=[]
    interval_test_errors=[]
    for t in range(len(errors)):
        err=errors[t]
        test_err=test_errors[t]
        temp=numpy.ones(len(err))
        temp_test=numpy.ones(len(err))
        temp[0]=numpy.nanmin([1,err[0]])
        temp_test[0]=numpy.nanmin([1,test_err[0]])
        for iter in range(1,len(err)):
            if numpy.nanmin([err[iter],temp[iter-1]])<temp[iter-1]:
                temp[iter]=err[iter]
                temp_test[iter]=test_err[iter]
            else:
                temp[iter]=temp[iter-1]
                temp_test[iter]=temp_test[iter-1]                
        min_errors.append(temp)
        min_test_errors.append(temp_test)
    min_iters=numpy.min([len(t) for t in min_errors])
    avg_interval_err = numpy.mean([t[:min_iters] for t in min_errors],axis=0)
    avg_interval_test_err = numpy.mean([t[:min_iters] for t in min_test_errors],axis=0)
    std_interval_err = numpy.std([t[:min_iters] for t in min_errors],axis=0)
    std_interval_test_err = numpy.std([t[:min_iters] for t in min_test_errors],axis=0)
    return len(errors), avg_interval_err, std_interval_err, avg_interval_test_err, std_interval_test_err

In [8]:
def calculate_percentiles(errors,test_errors,durations,intervals):
    min_errors=[]
    min_test_errors=[]
    sum_durations=[]
    interval_errors=[]
    interval_test_errors=[]
    for t in range(len(errors)):
        err=errors[t]
        test_err=test_errors[t]
        temp=numpy.ones(len(err))
        temp_test=numpy.ones(len(err))
        temp[0]=numpy.nanmin([1,err[0]])
        temp_test[0]=numpy.nanmin([1,test_err[0]])
        for iter in range(1,len(err)):
            if numpy.nanmin([err[iter],temp[iter-1]])<temp[iter-1]:
                temp[iter]=err[iter]
                temp_test[iter]=test_err[iter]
            else:
                temp[iter]=temp[iter-1]
                temp_test[iter]=temp_test[iter-1]                
        min_errors.append(temp)
        min_test_errors.append(temp_test)
    for dur in durations:
        temp = [sum(dur[0:(i+1)]) for i in range(len(dur))]
        sum_durations.append(temp)
    for trial in range(len(sum_durations)):
        min_err=min_errors[trial]
        min_test_err=min_test_errors[trial]
        sum_dur=sum_durations[trial]
        temp=[]
        temp_test=[]
        for t in intervals:
            indices=[ n for n,i in enumerate(sum_dur) if i<=t ]
            if len(indices)>0:
                temp.append(min_err[indices[-1]])
                temp_test.append(min_test_err[indices[-1]])
            else:
                temp.append(1)
                temp_test.append(1)
        interval_errors.append(temp)
        interval_test_errors.append(temp_test)
    err_median = numpy.median(interval_errors,axis=0)
    test_err_median = numpy.median(interval_test_errors,axis=0)
    err_high_qtr = numpy.percentile(interval_errors,75,axis=0,interpolation='midpoint')
    test_err_high_qtr = numpy.percentile(interval_test_errors,75,axis=0,interpolation='midpoint')
    err_low_qtr = numpy.percentile(interval_errors,25,axis=0,interpolation='midpoint')
    test_err_low_qtr = numpy.percentile(interval_test_errors,25,axis=0,interpolation='midpoint')
    return len(errors), err_median,test_err_median,err_high_qtr,test_err_high_qtr,err_low_qtr,test_err_low_qtr

In [9]:
def calculate_count(durations,intervals):
    sum_durations=[]
    interval_count=[]

    for dur in durations:
        temp = [sum(dur[0:(i+1)]) for i in range(len(dur))]
        sum_durations.append(temp)
    for trial in range(len(sum_durations)):
        sum_dur=sum_durations[trial]
        temp=[]
        for t in intervals:
            indices=[ n for n,i in enumerate(sum_dur) if i<=t ]
            if len(indices)>0:
                temp.append(len(indices))
            else:
                temp.append(0)
        interval_count.append(temp)
    avg_interval_count = numpy.mean(interval_count,axis=0)
    avg_interval_min = numpy.nanmin(interval_count,axis=0)
    avg_interval_max = numpy.nanmax(interval_count,axis=0)
    
    return avg_interval_count,avg_interval_min,avg_interval_max

In [31]:
def parse_pkl(files,smac_init=False):
    errors = []
    test_errors = []
    durations=[]
    
    for f in files:
        data=pickle.load(open(f,'rb'))
        if len(data['cv_endtime'])>0:
            error=[i['result'] for i in data['trials']]
            errors.append(error)
            test_error=[i['test_error'] for i in data['trials']]
            test_errors.append(test_error)
            duration=[data['cv_endtime'][0]-data['starttime'][0]]
            for t in range(1,len(data['cv_endtime'])):
                duration.append(data['cv_endtime'][t]-data['cv_endtime'][t-1])
            durations.append(duration)
    
    return_results=[errors,test_errors,durations]
    if smac_init:
        return return_results,[[t[0] for t in errors],[t[0] for t in test_errors],[t[0] for t in durations]]
    return return_results

In [25]:
#only include results on full training set
def parse_nvb_pkl(dataset,files,train_size_dictionary,smac_init=None):
    errors = []
    test_errors = []
    durations=[]
    
    for f in files:
        data=pickle.load(open(f,'rb'))
        max_train_size=train_size_dictionary[dataset]
        full_train_iters=numpy.where(numpy.asarray([t['train_size'] for t in data['trials']])==max_train_size)[0]
        if len(full_train_iters)<5:
            print f + ' has fewer than 5 arms trained on max sample sizes'
        if len(data['cv_endtime'])>0:
            error=[i['result'] for i in data['trials'] if i['train_size']==max_train_size]
            errors.append(error)
            test_error=[i['test_error'] for i in data['trials'] if i['train_size']==max_train_size]
            test_errors.append(test_error)
            duration=[data['cv_endtime'][full_train_iters[0]]-data['starttime'][0]]
            for t in range(1,len(full_train_iters)):
                duration.append(data['cv_endtime'][full_train_iters[t]]-data['cv_endtime'][full_train_iters[t-1]])
            durations.append(duration)
    
    if smac_init is not None:
        for t in range(len(errors)):
            errors[t].insert(0,smac_init[0][t])
            test_errors[t].insert(0,smac_init[1][t])
            durations[t].insert(0,smac_init[2][t])
    return errors,test_errors,durations

In [26]:
#combine 3 random runs for 3 hours
def parse_random_pkl(files,smac_init=None):
    errors = []
    long_errors = []
    long_test_errors = []
    long_durations = []
    test_errors = []
    durations=[]
    
    for f in files:
        data=pickle.load(open(f,'rb'))
        if len(data['cv_endtime'])>0:
            error=[i['result'] for i in data['trials']]
            test_error=[i['test_error'] for i in data['trials']]
            duration=[data['cv_endtime'][0]-data['starttime'][0]]
            for t in range(1,len(data['cv_endtime'])):
                duration.append(data['cv_endtime'][t]-data['cv_endtime'][t-1])
            if len(error)>len(duration):
                error.pop()
                test_error.pop()
            errors.append(error)
            test_errors.append(test_error)
            durations.append(duration)   
    for i in range(len(errors)/3):
        long_errors.append(errors[3*i]+errors[3*i+1]+errors[3*i+2])
        long_test_errors.append(test_errors[3*i]+test_errors[3*i+1]+test_errors[3*i+2])
        long_durations.append(durations[3*i]+durations[3*i+1]+durations[3*i+2])
    errors=long_errors
    test_errors=long_test_errors
    durations=long_durations
    
    if smac_init is not None:
        for t in range(len(errors)):
            errors[t].insert(0,smac_init[0][t])
            test_errors[t].insert(0,smac_init[1][t])
            durations[t].insert(0,smac_init[2][t])
    
    return errors,test_errors,durations

In [265]:
#results={}
intervals=range(30,3601,30)
intervals_long=range(30,10801,30)
results={}
smac_init={}
for dataset in datasets_keep:
    print dataset
    smac_files= glob.glob('../open_ml_'+str(dataset)+'_*smac*/*smac_2_06*.pkl')
    hyperopt_files=glob.glob('../open_ml_'+str(dataset)+'_*hyperopt*/*hyperopt_august2013_mod.pkl')
    random_files=glob.glob('../open_ml_'+str(dataset)+'_*random*/*random_hyperopt_august2013_mod.pkl')
    hyperopt_files=[h for h in hyperopt_files if h not in random_files]
    nvb_files=glob.glob('../open_ml_'+str(dataset)+'_*nvb_hyperband*/*nvb_hyperband.pkl')
    smac_files.sort()
    hyperopt_files.sort()
    random_files.sort()
    nvb_files.sort()
    results[dataset]={}
    #data=pickle.load(open(hyperopt_files[0],'rb'))
    #dataset_max_train_size[dataset]=numpy.nanmax([t['train_size'] for t in data['trials']])
    if len(smac_files)>0:
        dataset_results,dataset_init=parse_pkl(smac_files,True)
        results[dataset]['smac']=dataset_results
        smac_init[dataset]=dataset_init
    if len(hyperopt_files) > 0:
        results[dataset]['hyperopt']=parse_pkl(hyperopt_files)
    if len(random_files)>0:
        results[dataset]['random']=parse_random_pkl(random_files,dataset_init)
    if len(nvb_files)>0:
        results[dataset]['nvb']=parse_nvb_pkl(dataset,nvb_files,dataset_max_train_size,dataset_init)

2
3
4
5
8
9
10
11
12
13
14
15
16
18
20
21
22
23
24
28
29
30
31
32
33
35
36
37
39
40
41
42
43
47
48
49
50
52
53
54
55
57
58
59
60
2073
2074
2079
3022


In [266]:
filename='../all_results_smac_init.pkl'
pickle.dump(results,open(filename,'wb'))
#filename='dataset_max_train_size.pkl'
#pickle.dump(dataset_max_train_size,open(filename,'wb'))

In [284]:
results=pickle.load(open('./all_results.pkl','rb'))
#Calculate random average train time
dataset_avg_time={}
for d in results.keys():
    sum_dur=results[d]['random'][2]
    durations = []
    for t in range(len(sum_dur)):
        durations=durations + sum_dur[t]
    dataset_avg_time[d]=numpy.nanmean(durations)


In [285]:
#Calculate normalized interval errors
normalize=False
if normalize:
    max_iters=7200/int(numpy.min([dataset_avg_time[d] for d in dataset_avg_time.keys()]))
    intervals=range(1,max_iters+1)
    long_intervals=range(1,3*max_iters+3)
else:
    intervals=range(30,7201,30)
    long_intervals=range(30,21601,30)
condensed_results={}
searchers=results[2].keys()
for d in results.keys():
    condensed_results[d]={}
    for s in searchers:
        if s=='random':
            condensed_results[d][s]=calculate_avg(results[d][s][0],results[d][s][1],results[d][s][2],long_intervals,d)    
        else:
            condensed_results[d][s]=calculate_avg(results[d][s][0],results[d][s][1],results[d][s][2],intervals,d)    
        #Results append rf_baseline
    trials=condensed_results[d]['smac'][0]
    val_error = [numpy.mean([t[0] for t in results[d]['smac'][0]])]*len(intervals)
    test_error = [numpy.mean([t[0] for t in results[d]['smac'][1]])]*len(intervals)
    variance=[0]*len(intervals)
    condensed_results[d]['rf_baseline']=[trial,val_error,variance,test_error,variance]

In [286]:
filename='./all_results_condensed.pkl'
pickle.dump(condensed_results,open(filename,'wb'))

In [191]:
#Script to find out last update
argmins=[]
times=[]
for d in results.keys():
    data=results[d]['random'][0]
    indices = [numpy.argmin(t) for t in data]
    argmins = argmins+[numpy.argmin(t) for t in data]
    times=times+ [numpy.sum(results[d]['random'][2][t][0:indices[t]])/dataset_avg_time[d] for t in range(len(data))]
max(times)

(10, array([ 0.4123188,  0.2735507,  0.2731884, ...,  0.2507247,  0.2507247,
         0.2507247]), array([ 0.29415205,  0.02442626,  0.02415334, ...,  0.00720992,
         0.00720992,  0.00720992]), array([ 0.4391304,  0.2869565,  0.2942029, ...,  0.2927535,  0.2927535,
         0.2927535]), array([ 0.28679908,  0.06666643,  0.0590655 , ...,  0.04797956,
         0.04797956,  0.04797956]))

In [10]:
################################### ALL LINES BELOW NO LONGER USED ########################################
################################### ALL LINES BELOW NO LONGER USED ########################################
################################### ALL LINES BELOW NO LONGER USED ########################################
################################### ALL LINES BELOW NO LONGER USED ########################################
to_fix=[]

for d in datasets_keep:
    for s in ['smac','hyperopt','random','nvb']:
        if s not in results[d]:
            to_fix.append([d,s,0])
        elif results[d][s][0] < 10:
            to_fix.append([d,s,results[d][s][0]])
        elif nanmin(results[d][s][1])==1:
            to_fix.append([d,s,1])
            
            

In [94]:
def chart_error_iter(datasets, validation,interval_size):
    searchers=['smac','hyperopt','random','nvb']
    if validation==True:
        ind = 1
        title='Best Validation Error for Dataset '
        filename='individual_charts_val_error_iter.pdf'
    else:
        ind = 3
        title='Test Error Corresponding to Best Validation Error for Dataset '
        filename='individual_charts_test_error_iter.pdf'
    pdf_file=PdfPages(filename)
    for d in datasets:
        x = range(interval_size-1,len(results[d]['smac'][ind]),interval_size)
        indices=x
        plt.plot(x,[results[d]['smac'][ind][i] for i in indices],color='r',label='SMAC')
        x = range(interval_size-1,len(results[d]['hyperopt'][ind]),interval_size)
        indices=x
        plt.plot(x,[results[d]['hyperopt'][ind][i] for i in indices],color='b',label='Hyperopt/TPE')
        x = range(interval_size-1,len(results[d]['nvb'][ind]),interval_size)
        indices=x
        plt.plot(x,[results[d]['nvb'][ind][i] for i in indices],color='y',label='nvb_batch')
        x = range(interval_size-1,len(results[d]['random'][ind]),interval_size)
        indices=x
        plt.plot(x,[results[d]['random'][ind][i] for i in indices],color='g',label='Random')
        plt.xlabel('# of Arms Trained on Full Training Data')
        plt.title(title+str(d))
        plt.legend(loc=1)
        pdf_file.savefig()
        plt.close()
    pdf_file.close()

In [55]:
chart_error_iter(full_datasets,True,1)
chart_error_iter(full_datasets,False,1)

In [99]:
import csv
myfile=open("to_fix.csv",'wb')
wr=csv.writer(myfile)
for i in to_fix:
    wr.writerow(i)
myfile.close()


In [12]:
to_fix_id=numpy.unique([t[0] for t in to_fix])
full_datasets=[d for d in datasets_keep if d not in to_fix_id]

In [455]:
trial_iters=[]
for d in full_datasets:
    temp=[results[d]['smac'][0],results[d]['hyperopt'][0],results[d]['random'][0],results[d]['nvb'][0]]
    trial_iters.append(temp)

In [61]:
def chart_error(datasets, validation,indices,errorbars=False):
    if validation==True:
        ind = 1
        title='Best Validation Error for Dataset '
        filename='individual_charts_val_error.pdf'
    else:
        ind = 3
        title='Test Error Corresponding to Best Validation Error for Dataset '
        filename='individual_charts_test_error.pdf'
    pdf_file=PdfPages(filename)
    x=[intervals[i] for i in indices]
    for d in datasets:
        if errorbars:
            plt.errorbar(x,[results[d]['smac'][ind][i] for i in indices],yerr=[results[d]['smac'][ind+1][i] for i in indices],marker='o',color='r',markeredgecolor='r',markerfacecolor='None',label='SMAC')
            plt.errorbar(x,[results[d]['hyperopt'][ind][i] for i in indices],yerr=[results[d]['hyperopt'][ind+1][i] for i in indices],marker='x',color='b',label='Hyperopt/TPE')
            plt.errorbar(x,[results[d]['nvb'][ind][i] for i in indices],yerr=[results[d]['nvb'][ind+1][i] for i in indices],color='y',marker='^',markeredgecolor='y',markerfacecolor='None',label='nvb_batch')
            plt.errorbar(x,[results[d]['random'][ind][i] for i in indices],yerr=[results[d]['random'][ind+1][i] for i in indices],marker='+',color='g',label='Random')
        else:
            plt.errorbar(x,[results[d]['smac'][ind][i] for i in indices],marker='o',color='r',markeredgecolor='r',markerfacecolor='None',label='SMAC')
            plt.errorbar(x,[results[d]['hyperopt'][ind][i] for i in indices],marker='x',color='b',label='Hyperopt/TPE')
            plt.errorbar(x,[results[d]['nvb'][ind][i] for i in indices],color='y',marker='^',markeredgecolor='y',markerfacecolor='None',label='nvb_batch')
            plt.errorbar(x,[results[d]['random'][ind][i] for i in indices],marker='+',color='g',label='Random')
        plt.xlabel('Time (s)')
        plt.title(title+str(d))
        plt.legend(loc=1)
        pdf_file.savefig()
        plt.close()
    pdf_file.close()

In [62]:
chart_error(full_datasets,True,range(120),False)
chart_error(full_datasets,False,range(120),False)

In [138]:
def chart_error_perc(datasets, validation,interval_size):
    searchers=['smac','hyperopt','random','nvb']
    if validation==True:
        ind = 0
        title='Median Best Validation Error for Dataset '
        filename='individual_charts_val_error_perc.pdf'
    else:
        ind = 1
        title='Median Test Error Corresponding to Best Validation Error for Dataset '
        filename='individual_charts_test_error_perc.pdf'
    pdf_file=PdfPages(filename)
    for d in datasets:
        x = range(interval_size-1,len(results[d]['smac'][ind+1]),interval_size)
        indices=x
        plt.errorbar(x,[results[d]['smac'][ind][i] for i in indices],marker='o',color='r',markeredgecolor='r',markerfacecolor='None',label='SMAC')
        plt.errorbar(x,[results[d]['hyperopt'][ind][i] for i in indices],marker='x',color='b',label='Hyperopt/TPE')
        plt.errorbar(x,[results[d]['nvb'][ind][i] for i in indices],color='y',marker='^',markeredgecolor='y',markerfacecolor='None',label='nvb_batch')
        plt.errorbar(x,[results[d]['random'][ind][i] for i in indices],marker='+',color='g',label='Random')
        plt.title(title+str(d))
        plt.legend(loc=1)
        pdf_file.savefig()
        plt.close()
    pdf_file.close()

In [139]:
chart_error_perc(full_datasets,False,5)

In [516]:
#data_rel=[]
data_abs=[]
index_avg=3
for dataset in full_datasets:
    n_iters=len(intervals)
    iter_errors = [[results[dataset]['smac'][index_avg][i],results[dataset]['hyperopt'][index_avg][i],results[dataset]['random'][index_avg][i],results[dataset]['nvb'][index_avg][i]] for i in range(n_iters)]
    iter_min = [min(i) for i in iter_errors]
    #iter_rel = [[j/iter_min[i] for j in iter_errors[i]] for i in range(n_iters-10)]
    iter_abs = [[j-iter_min[i] for j in iter_errors[i]] for i in range(n_iters)]
    for k in range(n_iters):
        ind2=2*k+1
        ind3=3*k+2
        if ind2 < 2*(n_iters):
            iter_abs[k].append(results[dataset]['random'][index_avg][2*k+1]-iter_min[k])
        #else:
        #    iter_abs[k].append(nanmin(results[dataset]['random'][index_avg])-iter_min[k])
        if ind3 < 3*(n_iters):
            iter_abs[k].append(results[dataset]['random'][index_avg][3*k+2]-iter_min[k])
        #else:
        #    iter_abs[k].append(nanmin(results[dataset]['random'][index_avg])-iter_min[k])
    #data_rel.append(iter_rel)
    data_abs.append(iter_abs)


In [517]:
#data_rel=numpy.array(data_rel)
data_abs=numpy.array(data_abs)
#data_rel_avg=mean(data_rel,axis=0)
data_abs_avg=mean(data_abs,axis=0)    
data_abs_std=std(data_abs,axis=0)   

In [518]:
title='Test'
pdf_file=PdfPages('aggregate_charts_test.pdf')
iters=intervals
results_index=3

#3plt.figure(figsize=(11,8.5))
#plt.plot(iters,data_rel_avg[:,0],'r--',label='SMAC')
#plt.plot(iters,data_rel_avg[:,1],'bs',label='Hyperopt/TPE')
#plt.plot(iters,data_rel_avg[:,2],'g^',label='Random')
#plt.legend(loc=2)
#plt.title('Relative Performance Versus Best Validation Error Averaged Over 51 Datasets')
#pdf_file.savefig()
#plt.close()

plt.figure(figsize=(11,8.5))
#plt.ylim(-0.005,0.01)
plt.errorbar(iters[9:n_iters:10],data_abs_avg[9:n_iters:10,0],yerr=data_abs_std[9:n_iters:10,0],color='r',linestyle='--',label='SMAC')
plt.errorbar(iters[9:n_iters:10],data_abs_avg[9:n_iters:10,1],yerr=data_abs_std[9:n_iters:10,1],color='b',marker='s',label='Hyperopt/TPE')
plt.errorbar(iters[9:n_iters:10],data_abs_avg[9:n_iters:10,2],yerr=data_abs_std[9:n_iters:10,2],color='g',marker='^',label='Random')
plt.errorbar(iters[9:n_iters:10],data_abs_avg[9:n_iters:10,3],yerr=data_abs_std[9:n_iters:10,3],color='y',marker='o',label='nvb_batch')
plt.errorbar(iters[9:n_iters:10],data_abs_avg[9:n_iters:10,4],color='g',marker='o',label='Random 2x Speedup')
plt.errorbar(iters[9:n_iters:10],data_abs_avg[9:n_iters:10,5],color='g',marker='*',label='Random 3x Speedup')
plt.title('Absolute Difference from Best '+title+' Error Averaged Over 47 Datasets')
plt.legend()
plt.xlabel('Time (s)')
pdf_file.savefig()
plt.close()

ind = numpy.arange(len(full_datasets))  # the x locations for the groups
width = 0.18       # the width of the bars

for t in [30, 60, 90, 120]:
    fig, ax = plt.subplots(figsize=(11,8.5))
    rects1 = ax.bar(ind,[results[key]['smac'][results_index][t-1] for key in full_datasets], width, color='r')
    rects2 = ax.bar(ind + width, [results[key]['hyperopt'][results_index][t-1] for key in full_datasets], width, color='b')
    rects3 = ax.bar(ind + width +width, [results[key]['random'][results_index][t-1] for key in full_datasets], width, color='g')
    rects4 = ax.bar(ind + width +width+width, [results[key]['nvb'][results_index][t-1] for key in full_datasets], width, color='y')

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Datasets')
    ax.set_title('Average '+title+' Error After '+str(t*30/60)+' Minutes')
    plt.xlim(0,len(full_datasets)+1)
    ax.legend((rects1[0], rects2[0],rects3[0],rects4[0]), ('SMAC','TPE','Random','nvb_batch'),loc=2)
    #plt.tight_layout()
    pdf_file.savefig()
    plt.close()

pdf_file.close()

[[4, 'smac', 9],
 [8, 'smac', 9],
 [34, 'smac', 8],
 [37, 'smac', 8],
 [40, 'smac', 8],
 [42, 'smac', 8],
 [47, 'smac', 6],
 [50, 'smac', 8],
 [52, 'smac', 9],
 [59, 'smac', 7]]

120

{2: 718.0,
 3: 2556.0,
 4: 45.0,
 5: 361.0,
 6: 16000.0,
 8: 276.0,
 9: 164.0,
 10: 118.0,
 11: 500.0,
 12: 1600.0,
 13: 228.0,
 14: 1600.0,
 15: 559.0,
 16: 1600.0,
 18: 1600.0,
 20: 1600.0,
 21: 1382.0,
 22: 1600.0,
 23: 1178.0,
 24: 6499.0,
 26: 10368.0,
 28: 4496.0,
 29: 552.0,
 30: 4378.0,
 31: 800.0,
 32: 8793.0,
 33: 432.0,
 34: 72.0,
 35: 292.0,
 36: 1848.0,
 37: 614.0,
 38: 268.0,
 39: 166.0,
 40: 171.0,
 41: 546.0,
 42: 244.0,
 43: 3680.0,
 47: 120.0,
 48: 242.0,
 49: 766.0,
 50: 235.0,
 52: 216.0,
 53: 676.0,
 54: 124.0,
 55: 348.0,
 57: 280.0,
 58: 4000.0,
 59: 120.0,
 60: 80.0,
 2073: 1187.0,
 2074: 5144.0,
 2078: 329.0,
 2079: 588.0,
 3022: 792.0}